In [1]:
import pandas as pd

In [2]:

transformed_users_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_users.csv'
designations_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/raw/designations.csv'
transformed_courses_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_courses.csv'
transformed_progress_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_progress.csv'


fact_users_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/.venv/mart/dimension/fact_users.csv'

transformed_users_df = pd.read_csv(transformed_users_file)
transformed_courses_df = pd.read_csv(transformed_courses_file)
designations_df = pd.read_csv(designations_file)
transformed_progress_df =  pd.read_csv(transformed_progress_file)

#print(transformed_users_df.dtypes)
#print(transformed_courses_df.dtypes)
#print(designations_df.dtypes)
 #.replace({'m': 'M', 'f': 'F'})
 #beg_comp_rate avg rate at which the user has completed a beginner level courses.
#interm_comp_rate avg rate at which the user has completed a intermediate level courses.
#adv_comp_rate avg rate at which the user has completed a advanced level courses.
#total beg courses completed by the user  Fill with 0 for users with no completed courses
#total interm courses completed by the user 
#total advanced courses completed by the user 
 #you can use any of the tranformed tables as well. 
 #['active'] true or false given if they are in any courses currently, (less than 100percent completed )
 #['learning_speed'] based on if they have completed more than 50% of their assigned courses  quicker than duration of the course

In [3]:
# Rename 'user_id' to 'course_id' if that’s the intended change
transformed_courses_df.rename(columns={'user_id': 'course_id'}, inplace=True)


In [4]:

transformed_progress_df.rename(columns={'id': 'progress_id'}, inplace=True)


In [5]:
# set all employees role as employee and drop the role column,
transformed_users_df['role'] = 'employee'
transformed_users_df.drop(columns=['role'], inplace=True)

In [7]:
merged_progress_df = pd.merge(transformed_progress_df, 
                               transformed_courses_df[['course_id', 'title', 'proficiency_level']], 
                               left_on='courseId', 
                               right_on='course_id', 
                               how='left')

In [8]:
transformed_users_df['sex'] = transformed_users_df['sex'].replace({'m': 'M', 'f': 'F'})


In [10]:
# avg completion rates
completion_rates = merged_progress_df.groupby('userId').agg(
    beg_comp_rate=('percentage_completed', lambda x: x[merged_progress_df['proficiency_level'] == 'Beginner'].mean()),
    interm_comp_rate=('percentage_completed', lambda x: x[merged_progress_df['proficiency_level'] == 'Intermediate'].mean()),
    adv_comp_rate=('percentage_completed', lambda x: x[merged_progress_df['proficiency_level'] == 'Advanced'].mean()),
    total_beg_courses=('courseId', lambda x: (merged_progress_df['proficiency_level'] == 'Beginner').sum()),
    total_interm_courses=('courseId', lambda x: (merged_progress_df['proficiency_level'] == 'Intermediate').sum()),
    total_adv_courses=('courseId', lambda x: (merged_progress_df['proficiency_level'] == 'Advanced').sum())
).reset_index()



In [11]:
fact_users_df = pd.merge(transformed_users_df, transformed_progress_df, left_on='user_id', right_on='userId', how='left')
fact_users_df = pd.merge(fact_users_df, transformed_courses_df[['course_id', 'proficiency_level']], left_on='courseId', right_on='course_id', how='left')

In [13]:
from datetime import datetime

# Function to calculate the completion rate in days for completed courses
def calculate_completion_time(df, proficiency):
    # Filter by proficiency level and completed courses
    completed_courses = df[(df['proficiency_level'] == proficiency) & (df['percentage_completed'] == 100)]
    
    # Initialize a list to hold the completion times
    completion_times = []

    for user_id, group in completed_courses.groupby('user_id'):
        # Get the start and end dates for the user's courses in this difficulty
        start_dates = group[group['percentage_completed'] == 0][['updated_day', 'updated_month', 'updated_year']]
        end_dates = group[['updated_day', 'updated_month', 'updated_year']]

        # Check if there are completed courses
        if not start_dates.empty and not end_dates.empty:
            # Calculate the start date (first entry with 0%)
            start_date = datetime(year=start_dates['updated_year'].values[0],
                                  month=start_dates['updated_month'].values[0],
                                  day=start_dates['updated_day'].values[0])
            # Calculate the end date (first entry with 100%)
            end_date = datetime(year=end_dates['updated_year'].values[0],
                                month=end_dates['updated_month'].values[0],
                                day=end_dates['updated_day'].values[0])
            
            # Calculate the number of days to complete
            days_to_complete = (end_date - start_date).days
            completion_times.append((user_id, days_to_complete))

    # Create a DataFrame from the completion times
    completion_df = pd.DataFrame(completion_times, columns=['user_id', f'{proficiency.lower()}_comp_rate'])
    
    return completion_df

# Calculate completion rates for each proficiency level
beg_comp_rate_df = calculate_completion_time(fact_users_df, 'Beginner')
interm_comp_rate_df = calculate_completion_time(fact_users_df, 'Intermediate')
adv_comp_rate_df = calculate_completion_time(fact_users_df, 'Advanced')

# Print the completion rate DataFrames for debugging
print("Beginner Completion Rate DataFrame:\n", beg_comp_rate_df.head())
print("Intermediate Completion Rate DataFrame:\n", interm_comp_rate_df.head())
print("Advanced Completion Rate DataFrame:\n", adv_comp_rate_df.head())

# Merge the completion rate DataFrames back into fact_users_df
fact_users_df = fact_users_df.merge(beg_comp_rate_df, on='user_id', how='left', suffixes=('', '_beg'))
fact_users_df = fact_users_df.merge(interm_comp_rate_df, on='user_id', how='left', suffixes=('', '_interm'))
fact_users_df = fact_users_df.merge(adv_comp_rate_df, on='user_id', how='left', suffixes=('', '_adv'))

# Fill missing values with 0 for completion rates, using safe access
if 'beg_comp_rate' in fact_users_df.columns:
    fact_users_df['beg_comp_rate'] = fact_users_df['beg_comp_rate'].fillna(0)

if 'interm_comp_rate' in fact_users_df.columns:
    fact_users_df['interm_comp_rate'] = fact_users_df['interm_comp_rate'].fillna(0)

if 'adv_comp_rate' in fact_users_df.columns:
    fact_users_df['adv_comp_rate'] = fact_users_df['adv_comp_rate'].fillna(0)


Beginner Completion Rate DataFrame:
 Empty DataFrame
Columns: [user_id, beginner_comp_rate]
Index: []
Intermediate Completion Rate DataFrame:
 Empty DataFrame
Columns: [user_id, intermediate_comp_rate]
Index: []
Advanced Completion Rate DataFrame:
 Empty DataFrame
Columns: [user_id, advanced_comp_rate]
Index: []


In [16]:
# Save the final dimension table to CSV
fact_users_df.to_csv('C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/marts/fact/fact_users.csv', index=False)

print("fact_users.csv created successfully created ")

fact_users.csv created successfully created 
